In [167]:
from sklearn.model_selection import train_test_split
# import data_preprocessing.telco as data_loader
# import data_preprocessing.adult as data_loader
import data_preprocessing.credit as data_loader
import lib.regression as reg
import lib.log as log
from sklearn.metrics import accuracy_score
from numpy.random import choice
import numpy as np
np.random.seed(seed=42)
import importlib
importlib.reload(reg)
importlib.reload(data_loader)
importlib.reload(log)

<module 'lib.log' from '/mnt/604a8326-53f8-4a01-bed2-313518eb2318/LinuxData/Documents/Git/assignments/adaboost/lib/log.py'>

In [168]:
(X, Y) = data_loader.load(column_count=999)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)
X.shape

row count with class 1: 492
row count with class 0: 284315
0 classes are 577 times more than 1 classes
Dataframe columns: Index(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11',
       'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20', 'V21',
       'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount', 'Class'],
      dtype='object')
length of keeping attributes: 29
X_numpy shape: (20492, 30)
Y_numpy shape: (20492, 1)


(20492, 30)

In [169]:
K_ensembles = 5 # number of hypotheses
N_dataset_size = X_train.shape[0]
W_data_weights = np.full(N_dataset_size, 1 / N_dataset_size)
Z_model_weights = np.zeros((K_ensembles,))
models = []

In [170]:
for k in range(K_ensembles):
    sample_indices = choice(range(N_dataset_size), N_dataset_size,
              p=W_data_weights)
    X_sample_train = X_train[sample_indices]
    Y_sample_train = y_train[sample_indices]

    logistic_regression = reg.LogisticRegression()
    logistic_regression.fit(X_sample_train,
                            Y_sample_train,
                            number_of_epochs=1000,
                            learning_rate=0.01)

    print(f"Train accuracy of model {k}: {accuracy_score(y_train,logistic_regression.predict(X_train))}")

    print(f"Test accuracy of model {k}: {accuracy_score(y_test, logistic_regression.predict(X_test))}")
    models.append(logistic_regression)
    y_hypothesis = logistic_regression.predict(X_train)
    error = 0
    for j in range(N_dataset_size):
        if not y_hypothesis[j] == y_train[j]:
            error += W_data_weights[j]
    print(f"error: {error}")
    if error > 0.5:
        print("skipping ensemble")
        continue

    for j in range(N_dataset_size):
        if y_hypothesis[j] == y_train[j]:
            W_data_weights[j] += ((W_data_weights[j]*error)/(1-error))

    W_data_weights = W_data_weights / np.sum(W_data_weights)
    Z_model_weights[k] = np.log((1-error)/error)

Z_model_weights

Epoch 0. Loss= 0.369251909913769
Epoch 100. Loss= 0.023424814169668073
Epoch 200. Loss= 0.023424700444566612
Epoch 300. Loss= 0.02342467360468942
Epoch 400. Loss= 0.023424661538849353
Epoch 500. Loss= 0.02342465463396955
Epoch 600. Loss= 0.023424650130080982
Epoch 700. Loss= 0.02342464693722475
Epoch 800. Loss= 0.023424644538043764
Epoch 900. Loss= 0.02342464265519024
Train accuracy of model 0: 0.9945708534130422
Test accuracy of model 0: 0.9946328372773847
error: 0.005429146586957843
Epoch 0. Loss= 0.4199421321168953
Epoch 100. Loss= 0.019276110251847726
Epoch 200. Loss= 0.01830083843198876
Epoch 300. Loss= 0.01830062808447952
Epoch 400. Loss= 0.018300581853925028
Epoch 500. Loss= 0.01830055936442851
Epoch 600. Loss= 0.018300545740775557
Epoch 700. Loss= 0.01830053651393511
Epoch 800. Loss= 0.018300529814649608
Epoch 900. Loss= 0.01830052470997224
Train accuracy of model 1: 0.9947538583541755
Test accuracy of model 1: 0.9946328372773847
error: 0.005218475765455729
Epoch 0. Loss= 0.318

array([5.21052939, 5.25031778, 5.19767959, 5.27224168, 5.28931653])

In [171]:
weighted_predictions_test = np.zeros((K_ensembles, X_test.shape[0]))
for i in range(K_ensembles):
    Y_pred_test = models[i].predict(X_test)
    Y_pred_test = np.where(Y_pred_test == 0, -1, Y_pred_test) * Z_model_weights[i]
    weighted_predictions_test[i] = Y_pred_test.reshape((Y_pred_test.shape[0],))

prediction_sums = np.sum(weighted_predictions_test, axis=0)
predictions = np.where(prediction_sums > 0, 1, prediction_sums)
Y_pred_test = np.where(prediction_sums <= 0, 0, predictions)

print("testing accuracy: ")
log.report_performance(y_test, Y_pred_test)

testing accuracy: 
train accuracy: 0.9946328372773847
precision: 1.0
recall: 0.7708333333333334
f1 score: 0.8705882352941177
false discovery: 0.0
specificity: 1.0


In [172]:
weighted_predictions_train = np.zeros((K_ensembles, X_train.shape[0]))
for i in range(K_ensembles):
    Y_pred_train = models[i].predict(X_train)
    Y_pred_train = np.where(Y_pred_train == 0, -1, Y_pred_train) * Z_model_weights[i]
    weighted_predictions_train[i] = Y_pred_train.reshape((Y_pred_train.shape[0],))

prediction_sums = np.sum(weighted_predictions_train, axis=0)
predictions = np.where(prediction_sums > 0, 1, prediction_sums)
Y_pred_train = np.where(prediction_sums <= 0, 0, predictions)

print("training accuracy: ")
log.report_performance(y_train, Y_pred_train)



training accuracy: 
train accuracy: 0.994936863295309
precision: 0.9815384615384616
recall: 0.8055555555555556
f1 score: 0.8848821081830792
false discovery: 0.018461538461538463
specificity: 0.9996249296743139
